In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated

In [3]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [27]:
from typing import TypedDict
class State(TypedDict):
    messages: Annotated[list,add_messages]

def chatbot(state: State) -> State:
    response = [llm.invoke(state["messages"])]
    return ({"messages": response})

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)
graph = builder.compile()

In [29]:
messages = {"role": "user", "content": "Who walked on the moon first?"}
result = graph.invoke({"messages": [messages]})
print(result)

{'messages': [HumanMessage(content='Who walked on the moon first?', additional_kwargs={}, response_metadata={}, id='b7d0d6e1-1e90-4d6e-b17b-7da657eeac40'), AIMessage(content='Neil Armstrong was the first person to walk on the moon.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e9faa434-3d4d-4166-a334-284f161aad64-0', usage_metadata={'input_tokens': 7, 'output_tokens': 13, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})]}


In [ ]:
while True:
    in_message=input("You: ")
    if in_message.lower() in {"exit", "quit", "stop"}:
        print("Exiting the chatbot.")
        break
    if state is None:
        state: State = {
            "message": [{"role": "user", "content":in_message}]
        }
    else:
        state["messages"].append({"role":"user", "content":in_message})
    
    state= graph.invoke(state)
    print("Bot: ", state["messages"][-1]["content"])